In [1]:
import pandas as pd
import json
import copy

In [2]:
def from_json_to_df(json_file_path):
    """
    Convert the classification json file into a simple and usable dataframe. 
    We remove the key "species count" because it is useless here.
    Then to extract specific information of classification you can do df['000001.jpg']['track_info'] and then select the image
    """
    with open(json_file_path) as f:
        d = json.load(f)
        del d['species_count'] 
    df = pd.DataFrame.from_dict(d,orient='index').reset_index(level=0).rename(columns={"index": "source_image_cropped"})
    return df

In [3]:
csv_file_path = "../../0_database/tracking/inat_validation_data_with_sources_and_scores-20230204.csv" # contains the ground truth result for each cropped image
json_file_path = '../../0_database/tracking/set2_maxim-kent.json' # contains the classification result for each cropped image
df_gt = pd.read_csv(csv_file_path)
df_classification = from_json_to_df(json_file_path)

# Extract ground truth data

In [4]:
def extract_ground_truth_label(df_gt,df_classification):
    df = df_classification.merge(df_gt[['source_image_cropped','taxon_rank','taxon_name']],how='left',on='source_image_cropped')
    df.rename(columns={'taxon_rank':'taxon_rank_ground_truth',
                       'taxon_name':'taxon_name_ground_truth'}, inplace = True)
    return df

In [5]:
df_with_ground_truth = extract_ground_truth_label(df_gt,df_classification)

In [6]:
df_with_ground_truth

,source_image_cropped,region,date,track_id,prediction,track_info,taxon_rank_ground_truth,taxon_name_ground_truth
0,000001.jpg,Quebec,2022_07_31,2.0,"[[Choristoneura parallela, 29], [Argyrotaenia ...","[[20220731233507-00-91.jpg, 2432, 1485, 2566, ...",genus,Pandemis
1,000002.jpg,Quebec,2022_07_31,3.0,"[[Grammia virguncula, 33], [Martyringa latipen...","[[20220731233507-00-91.jpg, 3389, 1064, 3522, ...",genus,Olethreutes
2,000003.jpg,Quebec,2022_07_31,4.0,"[[Idia denticulalis, 31], [Metalectra discalis...","[[20220731233507-00-91.jpg, 600, 287, 751, 449...",genus,Idia
3,000004.jpg,Quebec,2022_07_31,6.0,"[[Agonopterix robiniella, 23], [Crambidia pall...","[[20220731233507-00-91.jpg, 1689, 740, 1780, 8...",tribe,Sparganothini
4,000005.jpg,Quebec,2022_07_31,7.0,"[[Archips cerasivorana, 25], [Rhynchagrotis cu...","[[20220731233507-00-91.jpg, 3482, 2052, 3649, ...",genus,Apotomis
...,...,...,...,...,...,...,...,...
995,000996.jpg,Quebec,2022_08_02,132.0,"[[Dioryctria disclusa, 21], [Acleris albicoman...","[[20220802024729-00-52.jpg, 3737, 1790, 3859, ...",genus,Acrobasis
996,000997.jpg,Quebec,2022_08_02,131.0,"[[Phaeoura quernaria, 38], [Thyris maculata, 1...","[[20220802024801-00-52.jpg, 1896, 978, 2237, 1...",species,Blepharomastix ranalis
997,000998.jpg,Quebec,2022_08_02,134.0,"[[Macaria granitata, 26], [Harrisimemna trisig...","[[20220802024638-00-52.jpg, 1295, 1285, 1543, ...",genus,Macaria
998,000999.jpg,Quebec,2022_08_02,137.0,"[[Eufidonia discospilata, 15], [Epelis truncat...","[[20220802024638-00-52.jpg, 430, 247, 537, 365...",tribe,Sparganothini


# Create df for evaluation and apply smoothing strategies

In [7]:
def smoothing_max_value(df):
    # ex: [['20220731233507-00-91.jpg', 2432, 1485, 2566, 1642, 'Choristoneura parallela', 29], ['20220731232026-00-79.jpg', 1650, 1277, 1768, 1448, 'Archips argyrospila', 39]]
    #display(df)
    max_confidence_score = df["confidence"].max()
    max_confidence_index = df["confidence"].idxmax()
    new_label = df.iloc[max_confidence_index, df.columns.get_loc("label")]
    return max_confidence_score, new_label

def smoothing_most_frequent(df):
    display(df)
    if df.shape[0]>1:
        new_label = df.label.mode()[0]
        max_confidence_score = df.loc[df['label'] == new_label]['confidence']
    else:
        max_confidence_score, new_label = smoothing_max_value(df)
    print(max_confidence_score, new_label)
    return max_confidence_score, new_label

def smoothing_avg_max(df):
    return 1,'test'

def create_df_evaluation(df):
    # test = df_evaluation.loc['000001.jpg']['track_info']
    # smoothing_max_value(test)
    smoothing_labels_max = []
    scores_max = []
    smoothing_labels_most_frequent = []
    scores_most_frequent = []
    smoothing_labels_avg_max = []
    scores_avg_max = []
    for row in df['track_info'][:10]:
        df_row = pd.DataFrame(row, columns = ['img', 'x1', 'x2', 'y1', 'y2', 'label', 'confidence'])
        score_max, label_max = smoothing_max_value(df_row)
        scores_max.append(score_max)
        smoothing_labels_max.append(label_max)
        ##
        score_most_frequent, label_most_frequent = smoothing_most_frequent(df_row)
        scores_most_frequent.append(score_most_frequent)
        smoothing_labels_most_frequent.append(label_most_frequent)
        ##
        score_avg_max, label_avg_max = smoothing_avg_max(df_row)
        scores_avg_max.append(score_avg_max)
        smoothing_labels_avg_max.append(label_avg_max)
    df['smoothing_label_max'] = smoothing_labels_max
    df['score_max'] = scores_max
    df['smoothing_label_most_frequent'] = smoothing_labels_most_frequent
    df['score_most_frequent'] = scores_most_frequent
    df['smoothing_label_avg_max'] = smoothing_labels_avg_max
    df['score_avg_max'] = scores_avg_max
    return df

In [8]:
df_evaluation = create_df_evaluation(df_with_ground_truth)

,img,x1,x2,y1,y2,label,confidence
0,20220731233507-00-91.jpg,2432,1485,2566,1642,Choristoneura parallela,29
1,20220731232026-00-79.jpg,1650,1277,1768,1448,Archips argyrospila,39


1    39
Name: confidence, dtype: int64 Archips argyrospila


,img,x1,x2,y1,y2,label,confidence
0,20220731233507-00-91.jpg,3389,1064,3522,1230,Grammia virguncula,33
1,20220731232026-00-79.jpg,3254,1412,3443,1577,Holomelina laeta,36


0    33
Name: confidence, dtype: int64 Grammia virguncula


,img,x1,x2,y1,y2,label,confidence
0,20220731233507-00-91.jpg,600,287,751,449,Idia denticulalis,31


...

,img,x1,x2,y1,y2,label,confidence
0,20220731233507-00-91.jpg,467,2018,543,2132,Endothenia hebesana,16


16 Endothenia hebesana


,img,x1,x2,y1,y2,label,confidence
0,20220731233507-00-91.jpg,533,856,651,1001,Epelis truncataria,30
1,20220731232026-00-79.jpg,3,633,110,739,Apogeshna stenialis,37
2,20220731230656-00-67.jpg,586,456,721,574,Argyrotaenia quercifoliana,54
3,20220731235027-00-100.jpg,507,551,614,677,Elophila ekthlipsis,31
4,20220731225403-00-59.jpg,1685,164,1776,292,Polygrammate hebraeicum,57
5,20220731234335-00-96.jpg,3602,1331,3701,1453,Haploa confusa,43
6,20220731235615-00-102.jpg,3551,1076,3636,1197,Thyris sepulchralis,14
7,20220731225108-00-58.jpg,3814,1212,3900,1336,Haploa confusa,26
8,20220731231928-00-79.jpg,3092,1160,3164,1295,Phobetron pithecium,9
9,20220731235622-00-102.jpg,3486,1314,3555,1437,Psychomorpha epimenis,44


4     57
19    34
24    45
Name: confidence, dtype: int64 Polygrammate hebraeicum


ValueError: Length of values (10) does not match length of index (1000)

def create